In [1]:
import pandas as pd
import numpy as np
import util as u

data='/labs/mpsnyder/moqri/data/index/'

In [2]:
## tss from ezh binding in h9
ezh_h9='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz' ## hg38
tss=pd.read_table(ezh_h9,index_col=0,usecols=['Probe','Chromosome','Start','End','WT_EZH2','Strand'])
tss.columns=['chrom', 'chromStart', 'chromEnd','s','v']
tss=tss[(tss.chrom!='X') & (tss.chrom!='Y')]
tss=tss[tss.index.str.contains('001')]
tss=tss[~tss.index.str.contains('-AS')]
tss.index=tss.index.str.strip('-001_upstream')
print(tss.shape)
tss['d']=tss.chromEnd-tss.chromStart
tss['tss']=tss.chromStart+2500
tss['g']=tss.chrom.astype(int)*10**9+tss.tss
print(tss.d.sum()/1000000,'M')
tss.to_csv('files/tss38.csv')

/home/moqri/anaconda3/envs/stem/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(31335, 5)
156.675 M


In [3]:
# cpg from h9 bw
v='38'
bw_url='http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg'+v+'/Human_H9.meth.bw'
df=u.bw2df(bw_url,tss)

1.1.1.1.1.1.1.1.1.1.1.1.1.1.2.2.2.2.2.2.2.2.2.2.2.3.3.3.3.3.3.3.4.4.4.4.4.4.4.5.5.5.5.5.5.5.5.6.6.6.6.6.6.6.6.7.7.7.7.7.7.7.8.8.8.8.8.8.8.9.9.9.9.9.10.10.10.10.10.10.11.11.11.11.11.11.11.11.11.11.12.12.12.12.12.12.12.12.12.13.13.13.14.14.14.14.14.14.15.15.15.15.15.15.16.16.16.16.16.16.16.16.17.17.17.17.17.17.17.17.17.17.18.18.18.18.19.19.19.19.19.19.19.19.19.19.20.20.20.20.21.21.22.22.22.22.

/oak/stanford/scg/lab_mpsnyder/moqri/aging/index/util.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop([0,1],1,inplace=True)


(3153046, 1)


In [4]:
df['g']=df.index.str.split('_').str[0].astype(int)*10**9+df.index.str.split('_').str[1].astype(int)
tssl=(tss.chrom.astype(int)*10**9+tss.tss).values
df['tss']=df.g.apply(lambda y:u.take_closest(tssl, y))
df['d']=df.g-df.tss
df['dq']=pd.qcut(df.d,11)
print(df.shape)
df.to_csv(data+'wg/cpgs.csv')

(3153046, 5)
